In [73]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt

from sklearn.preprocessing import MinMaxScaler, StandardScaler


from patchify import patchify

In [16]:
data_dir = 'Semantic_segmentation_dataset/'
scaler = MinMaxScaler()
patch_size = 256
images = []
masks = []

In [89]:
def load_patch_images(data_dir):
    image_dataset = []
    for path, subdirs, files in os.walk(data_dir):

        directory_name = path.split(os.path.sep)[-1]
        if directory_name == 'images':
            images = os.listdir(path)
            for i, image_name in enumerate(images):  
                if image_name.endswith(".jpg"):
                
                    image = cv2.imread(path+"/"+image_name, 1)
                    X_shape = (image.shape[1]//patch_size)*patch_size
                    Y_shape = (image.shape[0]//patch_size)*patch_size

                    image = Image.fromarray(image)
                    image = image.crop((0 ,0, X_shape, Y_shape))
                    image = np.array(image)

                    patch_img = patchify(image, (patch_size, patch_size, 3), step=patch_size)

                    for i in range(patch_img.shape[0]):
                        for j in range(patch_img.shape[1]):
                            single_patch_img = patch_img[i,j,:,:]
                            single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape)                        
                            single_patch_img = single_patch_img[0]
                            image_dataset.append(single_patch_img)

    return np.array(image_dataset), single_patch_img
                    


In [90]:
def load_patch_masks(data_dir):
    image_dataset = []
    for path, subdirs, files in os.walk(data_dir):

        directory_name = path.split(os.path.sep)[-1]
        if directory_name == 'masks':
            images = os.listdir(path)
            for i, image_name in enumerate(images):  
                if image_name.endswith(".png"):
                
                    image = cv2.imread(path+"/"+image_name, 1)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    X_shape = (image.shape[1]//patch_size)*patch_size
                    Y_shape = (image.shape[0]//patch_size)*patch_size

                    image = Image.fromarray(image)
                    image = image.crop((0 ,0, X_shape, Y_shape))
                    image = np.array(image)

                    patch_img = patchify(image, (patch_size, patch_size, 3), step=patch_size)

                    for i in range(patch_img.shape[0]):
                        for j in range(patch_img.shape[1]):
                            single_patch_img = patch_img[i,j,:,:]
                            single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape)                        
                            single_patch_img = single_patch_img[0]
                            image_dataset.append(single_patch_img)

    return np.array(image_dataset) , single_patch_img

In [ ]:
images , single_patch_img = np.array(load_patch_images(data_dir))
images.shape

In [ ]:
masks , single_patch_mask = np.array(load_patch_masks(data_dir))
masks.shape


In [95]:
Building = '#3C1098'.lstrip('#')
Land = '#8429F6'.lstrip('#')
Road = '#6EC1E4'.lstrip('#') 
Vegetation =  'FEDD3A'.lstrip('#') 
Water = 'E2A929'.lstrip('#') 
Unlabeled = '#9B9B9B'.lstrip('#') 

Building = np.array(tuple(int(Building[i:i+2], 16) for i in (0, 2, 4))) # 60, 16, 152
Land = np.array(tuple(int(Land[i:i+2], 16) for i in (0, 2, 4))) #132, 41, 246
Road = np.array(tuple(int(Road[i:i+2], 16) for i in (0, 2, 4))) #110, 193, 228
Vegetation = np.array(tuple(int(Vegetation[i:i+2], 16) for i in (0, 2, 4))) #254, 221, 58
Water = np.array(tuple(int(Water[i:i+2], 16) for i in (0, 2, 4))) #226, 169, 41
Unlabeled = np.array(tuple(int(Unlabeled[i:i+2], 16) for i in (0, 2, 4))) #155, 155, 155


array([155, 155, 155])

In [ ]:
class_dict = pd.read_csv("class_dict.csv", index_col=False, skipinitialspace=True)
class_dict

In [94]:
label_names= list(class_dict.name)
label_codes = []
r= np.asarray(class_dict['r'])
g= np.asarray(class_dict['g'])
b= np.asarray(class_dict['b'])

h = np.array(tuple([r[1], g[1], b[1]]))
"""
for i in range(len(class_dict)):
    label_codes.append(tuple([r[i], g[i], b[i]]))


    
code2id = {v:k for k,v in enumerate(label_codes)}
id2code = {k:v for k,v in enumerate(label_codes)}

name2id = {v:k for k,v in enumerate(label_names)}
id2name = {k:v for k,v in enumerate(label_names)}
"""

h

array([132,  41, 246], dtype=int64)